In [1]:
import re

# Extract error types from: "conll14st-preprocessed.conll.ann" file
# Separate each word of a particular error type in its own list.
l1=[]
l4=[]
l5=[]
l6=[]
l7=[]
l11=[]
l14=[]
def extract_err_type(colnn_ann):
    f = open(colnn_ann, 'r')
    for line in f:
        parts = line.split()
        if(len(parts)>0):
            if(line[0:6]=="<TYPE>"):
                type_ = line[6:len(parts[0])-7]
            elif(line[0:12]=="<CORRECTION>"):
                corrected_word = line[12:len(line)-14] 
                if type_ =="Vt":
                    "Vt", line[12:len(line)-14]
                    l1.append(line[12:len(line)-14])   
                if type_ =="Vform":
                    l4.append(line[12:len(line)-14])
                if type_ =="SVA":
                    l5.append(line[12:len(line)-14])
                if type_ =="ArtOrDet":
                    l6.append(line[12:len(line)-14])
                if type_ =="Nn":
                    l7.append(line[12:len(line)-14])
                if type_ =="Prep":
                    l11.append(line[12:len(line)-14])
                if type_ =="Wform":
                    l14.append(line[12:len(line)-14])

In [2]:
# Pass in to the function, the path to conll14st-preprocessed.conll.ann in your system
extract_err_type("/Users/highsierra/Tech-Skills/Labortory/ML-Fundamentals/error_types_conll14/conll14st-preprocessed.conll.ann")

In [3]:
# Create a corresponding list containing the error type of each word
# Map between each word and its error type
ll1=[]
for i in range(len(l1)):
    ll1.append('Vt')

ll4=[]
for i in range(len(l4)):
    ll4.append('Vform')

ll5=[]
for i in range(len(l5)):
    ll5.append('SVA')

ll6=[]
for i in range(len(l6)):
    ll6.append('ArtOrDet')

ll7=[]
for i in range(len(l7)):
    ll7.append('Nn')

ll11=[]
for i in range(len(l11)):
    ll11.append('Prep')


ll14=[]
for i in range(len(l14)):
    ll14.append('Wform')

In [4]:
# Separate the words (input) from the error types (target)
import itertools

targets = list(itertools.chain(ll1, ll4, ll5, ll6, ll7, ll11, ll14))
words = list(itertools.chain(l1, l4, l5, l6, l7, l11, l14))

In [5]:
print(len(words))

21138


In [6]:
# Encode the input using glove embeddings   

from zeugma.embeddings import EmbeddingTransformer

glove = EmbeddingTransformer('glove') 
X = glove.transform(words)

Using TensorFlow backend.


In [7]:
print(X)
print(X.shape)

[[ 9.4602001e-04 -8.4354997e-02  1.9994000e-01 ...  2.7858001e-01
   2.0168000e-01 -7.1859002e-01]
 [ 5.7466000e-02  1.7890000e-01 -6.2778002e-01 ... -7.5557001e-02
   2.4191999e-01  2.2470001e-03]
 [-5.2309000e-01 -6.8026000e-01 -4.6072000e-01 ...  5.2221000e-01
   7.0643997e-01 -2.5231999e-01]
 ...
 [ 5.6629997e-01  2.5286999e-01 -1.1373000e+00 ...  5.4163003e-01
  -1.7024000e+00 -4.1539001e-01]
 [ 3.4362650e-01  9.2356995e-02 -6.9733500e-01 ... -3.7974998e-02
  -1.1478500e+00 -2.5225499e-01]
 [ 1.0357000e-01  5.4285997e-01 -8.8192999e-01 ... -4.9219000e-01
   1.3172001e-01 -8.6390001e-01]]
(21138, 25)


In [8]:
print(len(targets))

21138


In [9]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
ybin = lb.fit(targets)
lb.classes_

y = lb.transform(targets)
print(y)
print(y.shape)

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]
(21138, 7)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from keras.layers.embeddings import Embedding

In [12]:
X_train = X_train.reshape(-1, 1, 25)
X_test  = X_test.reshape(-1, 1, 25)
y_train = y_train.reshape(-1, 1, 7)
y_test = y_test.reshape(-1, 1, 7)

In [13]:
model = Sequential()
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [14]:
model.fit(X_train, y_train, epochs=25, batch_size=64)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/25
16910/16910 [==============================] - 13s 747us/step - loss: 1.2749 - accuracy: 0.5731
Epoch 2/25
16910/16910 [==============================] - 8s 465us/step - loss: 1.0106 - accuracy: 0.6443
Epoch 3/25
16910/16910 [==============================] - 7s 443us/step - loss: 0.9650 - accuracy: 0.6560
Epoch 4/25
16910/16910 [==============================] - 8s 446us/step - loss: 0.9424 - accuracy: 0.6658
Epoch 5/25
16910/16910 [==============================] - 8s 448us/step - loss: 0.9197 - accuracy: 0.6726
Epoch 6/25
16910/16910 [==============================] - 8s 455us/step - loss: 0.9071 - accuracy: 0.6737
Epoch 7/25
16910/16910 [==============================] - 8s 449us/step - loss: 0.8981 - accuracy: 0.6800
Epoch 8/25
16910/16910 [==============================] - 10s 585us/step - loss: 0.8857 - accuracy: 0.6827
Epoch 9/25
16910/16910 [==============================]

In [15]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 71.59%
